<a href="https://www.kaggle.com/code/vovanquangnbk/mds-train?scriptVersionId=147066188" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Setup

In [1]:
!pip -q install transformers
!pip -q install evaluate
!pip -q install rouge_score
!pip -q install networkx==2.7.0
!pip -q install scipy==1.8.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
scikit-image 0.21.0 requires networkx>=2.8, but you have networkx 2.7 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
esda 2.5.0 requires scipy>=1.9, but you have scipy 1.8.0 which is incompatible.
featuretools 1.27.0 requires scipy>=1.10.0, but you have scipy 1.8.0 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.

In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import torch
from torch import nn
import os
import itertools
from datetime import datetime
import time
import random
import pprint
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics

# Hugging Face
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# Networkx
import networkx as nx

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cpu


In [3]:
CFG = {
    'show_examples': True,
    'data_dir': '/kaggle/input/drowsy-eye-keypoints',
    'cluster_sent_n': 20,
    'seed': 719,
    'model_arch': "VietAI/vit5-base-vietnews-summarization",
    'source_len': 1024,
    'target_len': 512,
    'text_len': 80,
    'text_rank': 15,
    'epochs': 1,
    'train_bs': 2,
    'valid_bs': 2,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
}

In [4]:
data_dir = '/kaggle/input/vims-dataset/ViMs'
original_dir = os.path.join(data_dir, 'original')
summary_dir = os.path.join(data_dir, 'summary')

## Utils

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table()

def resetTable():
    global training_logger

    training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)
resetTable()

In [7]:
def read_txt(path, article_type, sent=False):
    content = []
    write_file = False
    with open(path) as f:
        for line in f:
            if article_type == "original":
                if line.lower().startswith("content"):
                    write_file = True
            else:
                write_file = True
            if write_file: 
                if line.rstrip():
                    content.append(line.rstrip())
    if sent:
        return content[1:]
    return " ".join(content[1:])
if CFG['show_examples']:
    path1 = '/kaggle/input/vims-dataset/ViMs/original/Cluster_001/original/10.txt'
    path2 = '/kaggle/input/vims-dataset/ViMs/summary/Cluster_001/0.gold.txt'
    print(read_txt(path2, article_type="summary", sent=True))

['Các chuyên gia hàng không lo ngại một hành động cố ý đã khiến máy bay biến mất khỏi màn hình radar.', 'Bộ trưởng Hàng không Ai Cập Sherif Fathy nghiêng về khả năng MS804 bị khủng bố nhiều hơn là trục trặc kỹ thuật.', 'Họ nghi ngờ một quả bom khiến máy bay nổ tung, sau đó rơi xuống biển.', 'Tổng thư ký Tổ chức Hiệp ước Bắc Đại Tây Dương Jens Stoltenberg cho biết sẽ hỗ trợ công tác tìm kiếm máy bay MS 804.', 'một cuộc điều tra quy mô lớn đang được tiến hành sau khi chiếc A320 biến mất.', 'Theo các nguồn tin, hải mảnh nhựa vỡ được tìm thấy có màu trắng và đỏ.', 'Những mảnh vỡ  được phát hiện ở gần khu vực nơi hệ thống định vị khẩn cấp trên máy bay đã phát tín hiệu trước đó.', '66 người được cho là đã chết khi chuyến bay MS804 biến mất.', 'Ezeddin Samar là nữ tiếp viên có mặt trên chuyến bay MS804.', 'Ngay sau sự cố, có người phát hiện ra cô từng đăng ảnh bên cạnh một máy bay bị chìm.']


## CSV File

In [8]:
def create_csv(data_dir):
    """
    Input: data_dir
    - dir format: data_dir/original/cluster/original/txt
    Output: csv
    """
    df = {'cluster':[], 'path':[]}
    for cluster in os.listdir(data_dir):
        file_type = data_dir[data_dir.rfind("/")+1:]
        if file_type == "original":
            f_path = os.path.join(data_dir, cluster, file_type)
        else:
            f_path = os.path.join(data_dir, cluster)
        for f in glob(f_path + '/*'):
            df['cluster'].append(cluster)
            df['path'].append(f)

    df = pd.DataFrame(df)
    df = df.groupby('cluster')['path'].apply(list).reset_index()
    return df

original_df = create_csv(original_dir)
original_df.columns = ['cluster', 'original_dir']

summary_df = create_csv(summary_dir)
summary_df.columns = ['cluster', 'summary_dir']

df = original_df.merge(summary_df, how='inner', on='cluster')
if CFG['show_examples']:
    print(len(df))
    print(len(df['cluster'].unique()))
    print(df.head())

300
300
       cluster                                       original_dir  \
0  Cluster_001  [/kaggle/input/vims-dataset/ViMs/original/Clus...   
1  Cluster_002  [/kaggle/input/vims-dataset/ViMs/original/Clus...   
2  Cluster_003  [/kaggle/input/vims-dataset/ViMs/original/Clus...   
3  Cluster_004  [/kaggle/input/vims-dataset/ViMs/original/Clus...   
4  Cluster_005  [/kaggle/input/vims-dataset/ViMs/original/Clus...   

                                         summary_dir  
0  [/kaggle/input/vims-dataset/ViMs/summary/Clust...  
1  [/kaggle/input/vims-dataset/ViMs/summary/Clust...  
2  [/kaggle/input/vims-dataset/ViMs/summary/Clust...  
3  [/kaggle/input/vims-dataset/ViMs/summary/Clust...  
4  [/kaggle/input/vims-dataset/ViMs/summary/Clust...  


## Dataset

In [9]:
class MyDataset(Dataset):

    def __init__(
        self, dataframe, model, tokenizer, source_len, target_len, source_dir="original_dir", target_dir="summary_dir"
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            model (transformers.model): Transformers model
            source_len (int): Max length of source text
            target_len (int): Max length of target text
        """
        self.tokenizer = tokenizer
        self.model = model
        self.df = dataframe
        self.source_len = source_len
        self.target_len = target_len
        self.source_dir = self.df[source_dir]
        self.target_dir = self.df[target_dir]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.df)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""
        
        ## Source text
        source_dir_list = self.source_dir[index]
        source_text = []
        for i in range(len(source_dir_list)):
            txt = read_txt(source_dir_list[i], article_type="original", sent=True)
            source_text += txt

        source_text = self.textrank_extractor(source_text, k=CFG['text_rank'])
        
        ## Target text
        target_id = random.choice([0,1])
        target_dir_list = self.target_dir[index]
        target_text = read_txt(target_dir_list[target_id], article_type="summary")
        
        ## Tokenize
        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.target_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_txt": source_text,
            "target_txt": target_text,
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_mask": target_mask.to(dtype=torch.long),
        }
    
    def textrank_extractor(self, text, k=CFG['text_rank']):
        ## Extract input ids
        input_ids = self.tokenizer(text,
                        max_length=CFG['text_len'],
                        pad_to_max_length=True,
                        truncation=True,
                        padding="max_length",
                        return_tensors="pt",
                       )['input_ids']
        input_ids  = input_ids.to(device)
        
        ## Calculate embeddings
        embeddings = []
        for i in range(0, len(input_ids), CFG['train_bs']):
            batch = input_ids[i:i+CFG['train_bs']]
            emb = self.model.base_model.encoder(batch, return_dict=True).last_hidden_state
            embeddings.append(emb.detach())
        embeddings = torch.concat(embeddings)

        ## Similarity matrix
        mat_size = embeddings.shape[0]
        sim_mat = torch.zeros([mat_size, mat_size])
        for i in range(mat_size):
            for j in range(mat_size):
                if i != j:
                    sent1 = embeddings[i,:].reshape(1,-1)
                    sent2 = embeddings[j,:].reshape(1,-1)
                    sim_mat[i][j] = F.cosine_similarity(sent1, sent2)
        sim_mat = sim_mat.detach().cpu().numpy()


        ## Page rank
        nx_graph = nx.from_numpy_array(sim_mat)
        scores = nx.pagerank(nx_graph)
        ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(text)), reverse=True)
        output = " ".join([ranked_sentences[i][1] for i in range(k)])

        return output.strip()

if CFG['show_examples']:
    tokenizer = AutoTokenizer.from_pretrained(CFG['model_arch'])
    model = AutoModelForSeq2SeqLM.from_pretrained(CFG['model_arch'])
    model = model.to(device)
    
    dataset = MyDataset(df, model, tokenizer, CFG['source_len'], CFG['target_len'])
    for i, output in enumerate(dataset):
        print("Source:")
        print("Source article: \n", output['source_txt'])
        print("Source input ids length: \n", len(output['source_ids']))
        print("Source input ids: \n", output['source_ids'])
        print("Source attention mask: \n", output['source_mask'])
        print("\n")
        print("Target:\n")
        print("Target text: \n", output['target_txt'])
        print("Target input ids length: \n", len(output['target_ids']))
        print("Target attention mask: \n", output['target_mask'])
        if i >= 0:
            break

Source:
Source article: 
 Bạn bè của Hess cho biết, ông ấy gần như không thể đáp được chuyến bay MS804 của EgyptAir vì bị mất hộ chiếu. Chiếc máy bay này được sản xuất năm 2003, được điều khiển bởi hai phi công đã có hàng ngàn giờ kinh nghiệm trong buồng lái. Sau đó, Phó chủ tịch EgyptAir, Ahmed Adel thừa nhận EgyptAir đưa ra thông tin sai và “công tác tìm kiếm và cứu hộ đang diễn ra”. Đây là lần thứ hai trong vòng chưa đầy một năm, IS đã thành công sử dụng bom hẹn giờ trên một máy bay chở khách liên quan đến Ai Cập. Báo cáo của các cơ quan sân bay Ai Cập cho biết, hàng hóa không chứa các vật liệu nguy hiểm hoặc bất cứ thứ gì khác thường. Phó Chủ tịch EgyptAir, ông Ahmed Adel, không loại trừ khả năng máy bay bị đặt bom hoặc bị đe dọa từ bên trong. AP đưa tin rằng, một học sinh đến từ Chad đang học tại một trường học ở Pháp, cũng là một trong số các hành khách được cho là đã thiệt mạng. Ngoài ra, Tổng thống Hollande cũng gửi lời chia sẻ tới gia đình những người trên máy bay, cũng như kh

## Train

In [10]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 20 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    console.print(training_logger)
    resetTable()

In [11]:
def validate(epoch, tokenizer, model, device, loader):
    """
    Function to evaluate model for predictions
    """
    model.eval()
    
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
        
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=CFG['target_len'],
                num_beams=10,
                repetition_penalty=2.5,
                length_penalty=3.0,
                early_stopping=True
            )
            
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            
            if _%20==0:
                console.print(f'Completed {_}')
            
            predictions.extend(preds)
            actuals.extend(target)

    # print ROUGE score
    rouge = evaluate.load('rouge')
    results = rouge.compute(predictions=predictions,
                            references=actuals)
    console.print("ROUGE: ", results)
    
    return predictions, actuals

In [12]:
def T5Trainer(dataframe, output_dir="/kaggle/working/"):

    """
    T5 trainer
    """
    # Set random seeds and deterministic pytorch for reproducibility
    seed_everything(CFG['seed'])

    # logging
    console.log(f"""[Model]: Loading {CFG["model_arch"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = AutoTokenizer.from_pretrained(CFG['model_arch'])

    # Defining the model
    model = AutoModelForSeq2SeqLM.from_pretrained(CFG['model_arch'])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Creation of Dataset and Dataloader
    global val_dataset
    
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=CFG["seed"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = MyDataset(
        train_dataset,
        model,
        tokenizer,
        CFG['source_len'], 
        CFG['target_len']
    )
    val_set = MyDataset(
        val_dataset,
        model,
        tokenizer,
        CFG['source_len'], 
        CFG['target_len']
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": CFG["train_bs"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": CFG["valid_bs"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=CFG["lr"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(CFG["epochs"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    global final_df
    console.log(f"[Initiating Validation]...\n")
    predictions, actuals = validate(0, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
    final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

## Main

In [13]:
if __name__ == '__main__':
    T5Trainer(df)

[10:24:27] [Model]: Loading VietAI/vit5-base-vietnews-summarization...                             ]8;id=294561;file:///tmp/ipykernel_20/4028163281.py\4028163281.py]8;;\:]8;id=856630;file:///tmp/ipykernel_20/4028163281.py#10\10]8;;\
                                                                                                                   

[10:24:32] [Data]: Reading data...                                                                 ]8;id=958538;file:///tmp/ipykernel_20/4028163281.py\4028163281.py]8;;\:]8;id=687007;file:///tmp/ipykernel_20/4028163281.py#20\20]8;;\
                                                                                                                   

FULL Dataset: (300, 3)

TRAIN Dataset: (240, 3)

TEST Dataset: (60, 3)

...

ROUGE: 
{
    'rouge1': 0.4538093696624766,
    'rouge2': 0.2482976416986129,
    'rougeL': 0.2969806842736079,
    'rougeLsum': 0.2964002684306203
}

[13:24:16] [Validation Completed.]                                                                 ]8;id=695407;file:///tmp/ipykernel_20/4028163281.py\4028163281.py]8;;\:]8;id=613094;file:///tmp/ipykernel_20/4028163281.py#93\93]8;;\
                                                                                                                   

[Model] Model saved @ /kaggle/working/model_files

[Validation] Generation on Validation data saved @ /kaggle/working/predictions.csv

[Logs] Logs saved @ /kaggle/working/logs.txt

## Print result examples

In [14]:
# Pick randomly a cluster index
rd_idx = random.randint(0,len(val_dataset))

In [15]:
## Source text
print("Source: \n")
source_dir_list = val_dataset["original_dir"][rd_idx]
source_text = []
for i in range(len(source_dir_list)):
    txt = read_txt(source_dir_list[i], article_type="original")
    print(txt)

Source: 

Ngày 24/5, Công an tỉnh Quảng Nam cho biết, vừa bắt khẩn cấp 2 đối tượng bán vàng giả gồm: Đặng Xuân Hoa Diễm (SN 1985, trú xã Tân Sơn, TP.Pleiku, tỉnh Gia Lai) và Phạm Ngọc Tuấn (SN 1994, trú huyện Chư Sê, Gia Lai). Thời gian qua, nhiều chủ tiệm vàng ở các huyện Núi Thành, Tiên Phước, Bắc Trà My… bị một số đối tượng mang vàng giả đến bán trót lọt. Khoảng 8h ngày 9/4, một người phụ nữ điều khiển xe mô tô không rõ BKS đến tiệm vàng V.K.L. của gia đình ông V.V.V. (thị trấn Tiên Kỳ, huyện Tiên Phước) bán một sợi dây chuyền vàng 1 lượng. Sau khi thử thấy không có gì đáng ngờ nên chủ tiệm mua sợi dây chuyền này với giá 31,3 triệu đồng. Đến ngày 21/4, chủ tiệm dùng hóa chất để kiểm tra sợi dây chuyền này thì phát hiện dây chuyền bằng bạc, được mạ vàng bên ngoài. Cũng trong ngày 9/4, tiệm vàng của gia đình bà V.T.H. (thị trấn Tiên Kỳ, huyện Tiên Phước) cũng có một đối tượng nữ mang 1 lượng vàng giả đến bán với giá 30 triệu đồng. Đến ngày 12/4, chủ tiệm mang sợi dây chuyền này đến ti

In [16]:
## Actual
print("Actual: \n")
final_df['Actual Text'][rd_idx]

Actual: 



'Theo lời khai ban đầu của các đối tượng, từ ngày 9/4-11/5, 2 đối tượng Diễm và Tuấn cùng với Phan Văn Hoàn (SN 1985, ngụ thôn 3, xã Biển Hồ, TP Pleiku, Gia Lai) thực hiện trót lọt 8 vụ lừa đảo bán vàng giả với số tiền chiếm đoạt hàng trăm triệu đồng. Được biết, các đối tượng Diễm, Tuấn và Hoàn đều không có nghề nghiệp, đi lang thang khắp nơi để lừa đảo. Trong đó, đối tượng Hoàn có 2 tiền án, đối tượng Tuấn có 1 tiền án. Thủ đoạn của bọn tội phạm này là giả vờ túng thiếu nên bán vàng cưới để thực hiện giao dịch. Trong đó, có vụ số tiền lên tới 187 triệu đồng, thấp nhất cũng là 31 triệu đồng. Công an tỉnh Quảng Nam thông báo ai là nạn nhân hoặc có thông tin về nhóm lừa đảo này đề nghị liên hệ với thiếu tá Võ Hồng Nam, Đội trưởng PC45, điện thoại: 0914.196.787 để cung cấp thông tin./.'

In [17]:
## Prediction
print("Prediction: \n")
final_df['Generated Text'][rd_idx]

Prediction: 



'Chiều 23/5, Phòng Cảnh sát Hình sự Công an tỉnh Quảng Nam cho biết, vừa bắt khẩn cấp nhóm đối tượng lừa đảo hàng trăm triệu đồng bằng thủ đoạn bán vàng giả cho các tiệm vàng. Theo điều tra, đối tượng Hoàn có 1 tiền án, Hoàn có 2 tiền án.'